# Project

In [1]:
using CSV, DataFrames, JuMP, Gurobi, Plots, LinearAlgebra, Dates, JSON

### Preprocessing

In [2]:
fires = CSV.read("data/wildfire_suppression.csv", DataFrame, header = true);
first(fires, 5)

Row,report_id,fire_id,area,date,longitude,latitude,from_date,crews_sent,total_crews_sent,max_crews_sent
,Int64,Int64,Float64,String31,Float64,Float64,String31,Float64,Float64,Float64
1,2714023,2714022,150.0,2015-05-06 10:15:00,-71.2519,43.7811,2015-05-05 09:30:00,45.0,123.0,45.0
2,2714037,2714022,275.0,2015-05-08 00:30:00,-71.2519,43.7811,2015-05-07 11:00:00,45.0,123.0,45.0
3,2714050,2714022,275.0,2015-05-09 00:30:00,-71.2519,43.7811,2015-05-08 11:00:00,33.0,123.0,45.0
4,2714066,2714022,275.0,2015-05-10 13:00:00,-71.2519,43.7811,2015-05-09 15:00:00,0.0,123.0,45.0
5,2714082,2714081,205.0,2015-05-07 07:15:00,-67.0125,44.7917,2015-05-06 20:30:00,21.0,37.0,21.0


In [3]:
function parse_date(date_string)
    try
        return DateTime(date_string, "yyyy-mm-dd HH:MM:SS.sss")
    catch
        return DateTime(date_string, "yyyy-mm-dd HH:MM:SS")
    end
end
fires.date = [parse_date(date_string) for date_string in fires.date]
fires_df = filter(row -> Dates.format(row.date, "yyyy-mm-dd") == "2017-08-01", fires);

In [4]:
Fires = Matrix(fires_df);
n_fires = 1:nrow(fires_df);
Surfaces = fires_df[:, :area];

### Distance feature

In [5]:
# Calculate Euclidian distance
function euclidean_distance(lat1, lon1, lat2, lon2)
    return sqrt((lat2 - lat1)^2 + (lon2 - lon1)^2)
end

# Calculate the Haversine distance
function haversine_distance(lat1, lon1, lat2, lon2)
    R = 6371.0 # Earth radius in kilometers
    dLat = deg2rad(lat2 - lat1)
    dLon = deg2rad(lon2 - lon1)
    lat1 = deg2rad(lat1)
    lat2 = deg2rad(lat2)

    a = sin(dLat/2)^2 + cos(lat1) * cos(lat2) * sin(dLon/2)^2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c
end

# Returns distance matrix of distances between fires
function distance_matrix(df, euclidian=1)
    n = size(df,1)
    dist_matrix = zeros(n, n)
    for i in 1:n
        for j in (i+1):n
            if euclidian == 1
                dist = euclidean_distance(df[i][:,:latitude][1], df[i][:,:longitude][1], df[j][:,:latitude][1], df[j][:,:longitude][1])
            else
                dist = haversine_distance(df[i][:,:latitude][1], df[i][:,:longitude][1], df[j][:,:latitude][1], df[j][:,:longitude][1])
            end 
            dist_matrix[i, j] = dist
            dist_matrix[j, i] = dist
        end
    end
    return dist_matrix
end; 


### Data ready

In [6]:
fires = CSV.read("data/wildfire_suppression.csv", DataFrame, header = true);
function parse_date(date_string)
    try
        return DateTime(date_string, "yyyy-mm-dd HH:MM:SS.sss")
    catch
        return DateTime(date_string, "yyyy-mm-dd HH:MM:SS")
    end
end
fires.date = [parse_date(date_string) for date_string in fires.date]
first(fires, 2)

Row,report_id,fire_id,area,date,longitude,latitude,from_date,crews_sent,total_crews_sent,max_crews_sent
,Int64,Int64,Float64,DateTime,Float64,Float64,String31,Float64,Float64,Float64
1,2714023,2714022,150.0,2015-05-06T10:15:00,-71.2519,43.7811,2015-05-05 09:30:00,45.0,123.0,45.0
2,2714037,2714022,275.0,2015-05-08T00:30:00,-71.2519,43.7811,2015-05-07 11:00:00,45.0,123.0,45.0


### Current test

In [8]:
alpha = 0.01

0.01

In [121]:
fires_on_august_1_2017 = filter(row -> Dates.format(row.date, "yyyy-mm-dd") == "2017-08-01", fires)

n_fighters = Int(floor(sum(fires_on_august_1_2017[:,:crews_sent])) / 34); # = 523
n_fires = nrow(unique(fires_on_august_1_2017, :fire_id));

grouped_fires = groupby(fires_on_august_1_2017, :fire_id)
demand = [sum(group[!,:max_crews_sent]) for group in grouped_fires]
demand = demand ./ 34

# Get assignements on freeze day 
initial_allocation = [sum(group[!,:crews_sent]) for group in grouped_fires];
initial_allocation = floor.(initial_allocation ./ 34)

assignements = zeros(Int(n_fighters), n_fires);
k, start = 1, 1
for f in initial_allocation
    num_ones = Int(f)
    assignements[start:num_ones, k] .= 1
    start = Int(f)+1
    k += 1
end  

Distances = distance_matrix(grouped_fires);

In [90]:
Distances[1,1]

0.0

In [105]:
alpha=5

5

In [122]:
n_fighters = Int(sum(fires_on_august_1_2017[:,:crews_sent]));
demand = [];
fires_list = [];
days = ["2017-08-01", "2017-08-02", "2017-08-03"];

for day in days
    fires_on_day_i = filter(row -> Dates.format(row.date, "yyyy-mm-dd") == day, fires)
    
    push!(fires_list, nrow(unique(fires_on_day_i, :fire_id)));
    
    # Group by fire_id and calculate sum of max_crews_sent for each group
    grouped_fires = groupby(fires_on_day_i, :fire_id)
    sum_max_crews_sent = [sum(group[!,:max_crews_sent]) for group in grouped_fires]

    # Push the array of sums to the demand array
    push!(demand, sum_max_crews_sent)
end

In [116]:

model = Model(Gurobi.Optimizer)
time=4
@variable(model, Y[k1=1:n_fires, k2=1:n_fires, t=1:time]>=0)
@variable(model, pos_cost[k=1:n_fires, t=1:time]>=0)
@constraint(model, pos_cost_con[k=1:n_fires, t=1:time],
            pos_cost[k, t] >= demand[1][k] - sum(sum(Y[k1,k,t_] for k1=1:n_fires) for t_=1:t))

@objective(model, Min, sum(pos_cost[k, t] for k=1:n_fires, t=1:time) + 
                        alpha * sum(Y[k1,k2,t] * Distances[k1,k2] for k1=1:n_fires, k2=1:n_fires, t=1:time))


@constraint(model, initial, Y[:,:,1].==Y_init)
@constraint(model, constraint_fighters[t=1:time-1, k=1:n_fires],
            sum(Y[k1,k,t] for k1=1:n_fires)==sum(Y[k,k2,t+1] for k2=1:n_fires))

set_optimizer_attribute(model, "TimeLimit", 60.0)
optimize!(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22


LoadError: MethodError: no method matching (::Colon)(::Int64, ::Vector{Any})

[0mClosest candidates are:
[0m  (::Colon)(::T, ::Any, [91m::T[39m) where T<:Real
[0m[90m   @[39m [90mBase[39m [90m[4mrange.jl:45[24m[39m
[0m  (::Colon)(::A, ::Any, [91m::C[39m) where {A<:Real, C<:Real}
[0m[90m   @[39m [90mBase[39m [90m[4mrange.jl:10[24m[39m
[0m  (::Colon)(::T, ::Any, [91m::T[39m) where T
[0m[90m   @[39m [90mBase[39m [90m[4mrange.jl:44[24m[39m
[0m  ...


In [110]:
690

690

In [ ]:
demand fixe : 887 891

In [72]:
value.(Y)

51×51×10 Array{Float64, 3}:
[:, :, 1] =
 0.0   0.0   0.0  0.0   0.0   0.0  0.0  …  0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  11.0   0.0  0.0   0.0   0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0   0.0  77.0  0.0   0.0   0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0   0.0   0.0  1.0   0.0   0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0   0.0   0.0  0.0  17.0   0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0   0.0   0.0  0.0   0.0  18.0  0.0  …  0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0   0.0   0.0  0.0   0.0   0.0  7.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0   0.0   0.0  0.0   0.0   0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0   0.0   0.0  0.0   0.0   0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0   0.0   0.0  0.0   0.0   0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0   0.0   0.0  0.0   0.0   0.0  0.0  …  0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0   0.0   0.0  0.0   0.0   0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0   0.0   0.0  0.0   

In [66]:
580 

580

In [10]:
n_groups = n_fighters
model = Model(Gurobi.Optimizer)

# Redefine the variables with groups instead of individual fighters
@variable(model, X[g=1:n_groups, k=1:n_fires, t=1:3], binary = true)
@variable(model, Y[g=1:n_groups, k1=1:n_fires, k2=1:n_fires, t=2:3], binary = true)

# New variables for positive parts of costs
@variable(model, pos_cost[k=1:n_fires, t=1:3] >= 0)

# Constraints for new variables
@constraint(model, pos_cost_con[k=1:n_fires[1], t=1:3],
            pos_cost[k, t] >= demand[k] - sum(sum(X[g,k,t_] for g=1:n_groups) for t_=1:t))

@objective(model, Min, sum(pos_cost[k, t] for k=1:n_fires, t=1:3) + 
                        alpha * sum(Y[g,k1,k2,t] * Distances[k1,k2] for g=1:n_groups, k1=1:n_fires, k2=1:n_fires, t=2:3))

# Adjusted constraints for groups
@constraint(model, constraint_fighters1[t=1:3, g=1:n_groups],
            sum(X[g,k,t] for k=1:n_fires) == 1)
@constraint(model, constraint_Y[t=2:3, k1=1:n_fires, k2=1:n_fires, g=1:n_groups],
            X[g,k1,t-1] + X[g,k2,t] - 1 <= Y[g,k1,k2,t])

set_optimizer_attribute(model, "TimeLimit", 60.0)
optimize!(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Set parameter TimeLimit to value 60
Set parameter TimeLimit to value 60
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2722368 rows, 2800818 columns and 8402148 nonzeros
Model fingerprint: 0x7d363db7
Variable types: 153 continuous, 2800665 integer (2800665 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-04, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-02, 3e+02]
Found heuristic solution: objective 2109.1807806
Presolve removed 53346 rows and 53346 columns (presolve time = 5s) ...
Presolve removed 53346 rows and 53346 columns (presolve time = 10s) ...
Presolve removed 53346 rows and 53346 columns (presolve time = 15s) ...
Presolve removed 53295 ro

In [11]:
X_current = JuMP.value.(X);
Y_current = JuMP.value.(Y);

In [12]:
# Initialize Y1_g_k1_k2_1 tensor
Y1_g_k1_k2_1 = zeros(Int, n_groups, n_fires, n_fires)

# Fill the tensor based on group movements from day 0 to day 1
for g in 1:n_groups
    for k1 in 1:n_fires
        for k2 in 1:n_fires
            if assignements[g, k1] == 1 && X_current[g, k2, 1] == 1
                Y1_g_k1_k2_1[g, k1, k2] = 1
            end
        end
    end
end

# Get all movements from day 0 to day 3
Y_total = zeros(Int, n_groups, n_fires, n_fires, 3);
Y_total[:,:,:,1] = Y1_g_k1_k2_1;
Y_total[:,:,:,2] = Y_current[:,:,:,2];
Y_total[:,:,:,3] = Y_current[:,:,:,3];

In [13]:
function cost_function(X,Y,Distances,demand)
    pos_cost_component = sum(
        max(0, demand[k] - sum(sum(X[g, k, t_] for g in 1:n_groups) for t_ in 1:t))
        for k in 1:n_fires, t in 1:3
    )
    
    movement_cost_component = alpha * sum(
        Y[g, k1, k2, t] * Distances[k1, k2]
        for g in 1:n_groups, k1 in 1:n_fires, k2 in 1:n_fires, t in 1:3
    )
    
    total_cost = pos_cost_component + movement_cost_component
    
    println("Total cost of the solution is: $total_cost")

    return total_cost
end

cost_function (generic function with 1 method)

In [14]:
cost_model_1 = cost_function(X_current,Y_total,Distances,demand);

Total cost of the solution is: 2138.4455589658132


In [15]:
# WRITE MOVEMENTS Y TO A CSV FILE (to handle in Python for visualisation)

# Arrays to hold the data for each column
group_data = Int[]
from_fire_data = Int[]
to_fire_data = Int[]
day_data = Int[]
distance_data = Float64[]

# Collect data
for t in 1:3, g in 1:n_groups, k1 in 1:n_fires, k2 in 1:n_fires
    if Y_total[g, k1, k2, t] == 1
        push!(group_data, g)
        push!(from_fire_data, k1)
        push!(to_fire_data, k2)
        push!(day_data, t)
        push!(distance_data, Distances[k1, k2])
    end
end

# Create the DataFrame from the collected data
movement_data = DataFrame(Group = group_data, 
                          From_Fire = from_fire_data, 
                          To_Fire = to_fire_data, 
                          Day = day_data, 
                          Distance = distance_data)

# Write to CSV
CSV.write("fire_movement_data.csv", movement_data)

"fire_movement_data.csv"

In [16]:
# Get fire coordinates
fire_coords = []
for (k, coord) in enumerate(grouped_fires)
    push!(fire_coords,(lon=coord[1,:longitude],lat=coord[1,:latitude]))
end

In [17]:
# Assuming Y1 is your variable
json_coord = JSON.json(fire_coords)

# Write to a file
open("coord.json", "w") do f
    write(f, json_coord)
end

1702

### Model Alex 2

In [18]:
fires_on_august_1_2017_sorted = sort(fires_on_august_1_2017, :area, rev=true)
first(fires_on_august_1_2017_sorted,3)

Row,report_id,fire_id,area,date,longitude,latitude,from_date,crews_sent,total_crews_sent,max_crews_sent
,Int64,Int64,Float64,DateTime,Float64,Float64,String31,Float64,Float64,Float64
1,7269653,7269467,270723.0,2017-08-01T02:00:00,-107.758,47.3183,2017-07-31 14:00:00,179.0,5915.0,656.0
2,7170884,7170139,81826.0,2017-08-01T00:45:00,-120.207,37.6194,2017-07-31 18:00:00,1627.0,127933.0,5128.0
3,7170401,7170139,81826.0,2017-08-01T13:30:00,-120.207,37.6194,2017-08-01 08:00:00,992.0,127933.0,5128.0


In [19]:
# Group by fire_id and calculate demand
grouped_fires_sorted = groupby(fires_on_august_1_2017_sorted, :fire_id)
demand_sorted = [sum(group[!, :max_crews_sent]) for group in grouped_fires_sorted]
demand_sorted = Int.(floor.(demand_sorted ./ 34))
n_fighters = Int(floor(sum(fires_on_august_1_2017_sorted[:,:crews_sent])) / 34); # = 523
n_fires = nrow(unique(fires_on_august_1_2017_sorted, :fire_id));

# Get assignements on freeze day 
initial_allocation_sorted = [sum(group[!,:crews_sent]) for group in grouped_fires_sorted];
initial_allocation_sorted = floor.(initial_allocation_sorted ./ 34)

assignements_sorted = zeros(Int(n_fighters), n_fires);
k, start = 1, 1
for f in initial_allocation_sorted
    num_ones = Int(f)
    assignements_sorted[start:num_ones, k] .= 1
    start = Int(f)+1
    k += 1
end  

Distances_sorted = distance_matrix(grouped_fires_sorted);

n_groups = n_fighters

523

In [20]:
X = zeros(n_groups, n_fires, 3)
# Treat big fires first
for t=1:3
    remaining_fighters = n_groups
    pointer = 1
    for (k, group) in enumerate(grouped_fires_sorted)
        if demand_sorted[k] > 0  # Check if the fire still needs treatment
            allocated_fighters = min(demand_sorted[k], remaining_fighters)
            X[pointer:pointer + allocated_fighters-1, k, t] .= 1
            demand_sorted[k] -= allocated_fighters  # Update remaining demand
            remaining_fighters -= allocated_fighters
            pointer += allocated_fighters
        end
        if remaining_fighters <= 0
            break  # Move to the next day if no firefighters are left
        end
    end
end

Y = zeros(n_groups, n_fires, n_fires, 3);
# Fill the tensor Y based on group movements
for t in 1:3 
    for g in 1:n_groups  # Each group
        for k1 in 1:n_fires  # Each fire on day t-1
            for k2 in 1:n_fires  # Each fire on day t
                if t == 1
                    if assignements_sorted[g, k1] == 1 && X[g, k2, t] == 1
                        Y[g, k1, k2, t] = 1
                    end
                # Check if group g moved from fire k1 to k2
                else
                    if X[g, k1, t-1] == 1 && X[g, k2, t] == 1
                        Y[g, k1, k2, t] = 1
                    end
                end
            end
        end
    end
end

In [25]:
X

523×51×3 Array{Float64, 3}:
[:, :, 1] =
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0

In [21]:
total_cost_alex2 = cost_function(X,Y,Distances_sorted,demand_sorted)

Total cost of the solution is: 69.82633222669476


69.82633222669476

In [22]:
# WRITE MOVEMENTS Y TO A CSV FILE (to handle in Python for visualisation)

# Arrays to hold the data for each column
group_data = Int[]
from_fire_data = Int[]
to_fire_data = Int[]
day_data = Int[]
distance_data = Float64[]

# Collect data
for t in 1:3, g in 1:n_groups, k1 in 1:n_fires, k2 in 1:n_fires
    if Y[g, k1, k2, t] == 1
        push!(group_data, g)
        push!(from_fire_data, k1)
        push!(to_fire_data, k2)
        push!(day_data, t)
        push!(distance_data, Distances_sorted[k1, k2])
    end
end

# Create the DataFrame from the collected data
movement_data_alex2 = DataFrame(Group = group_data, 
                          From_Fire = from_fire_data, 
                          To_Fire = to_fire_data, 
                          Day = day_data, 
                          Distance = distance_data)

# Write to CSV
CSV.write("fire_movement_data_alex2.csv", movement_data_alex2)

"fire_movement_data_alex2.csv"

In [23]:
fire_coords_alex2 = []
for (k, coord) in enumerate(grouped_fires_sorted)
    push!(fire_coords_alex2,(lon=coord[1,:longitude],lat=coord[1,:latitude]))
end

In [24]:
#using JSON

# Assuming Y1 is your variable
json_coord_alex2 = JSON.json(fire_coords_alex2)

# Write to a file
open("coord_alex2.json", "w") do f
    write(f, json_coord_alex2)
end

1702

### First model (too many variables)

In [ ]:
model = Model(Gurobi.Optimizer);
@variable(model, X1[i=1:n_fighters, k=1:n_fires, t=1:3], binary = true)

@variable(model, Y1[i=1:n_fighters, k1=1:n_fires, k2=1:n_fires, t=2:3], binary = true)

# New variables for positive parts of costs
@variable(model, pos_cost[k=1:n_fires, t=1:3] >= 0)

# Constraints for new variables
@constraint(model, pos_cost_con[k=1:n_fires, t=1:3], pos_cost[k, t] >= demand[k] - sum(X1[i,k,t] for i=1:n_fighters))

# Adjusted objective function
@objective(model, Min, sum(pos_cost[k, t] for k=1:n_fires, t=1:3) +
                        sum(Y1[i,k1,k2,t] * Distances[k1,k2] for i=1:n_fighters, k1=1:n_fires, k2=1:n_fires, t=2:3))

@constraint(model, constraint_fighters1[t=1:3, i in n_fighters], sum(X1[i,k,t] for k in n_fires) == 1);

@constraint(model, constraint_Y1[t=2:3, k1=1:n_fires, k2=1:n_fires, i=1:n_fighters],  X1[i,k1,t-1] +X1[i,k2,t] - 1 <= Y1[i,k1,k2,t]);

#@constraint(model, constraint_Y1_day1[t=1, k1=1:n_fires[1], k2=1:n_fires[1], i=1:n_fighters],  assignements[i,k1] +X1[i,k2,t] - 1 <= Y1[i,k1,k2,t]);

#@objective(model, Min, sum(sum(demand[1][k] - sum(X1[i,k,t] for i in n_fighters) for k in n_fires[1]) for t=1:3) + 
#                        sum(sum(sum(sum(Y1[i,k1,k2,t] * Distances[k1,k2] for i in n_fighters) for k1 in n_fires[1]) for k2 in n_fires[1]) for t=2:3));

set_optimizer_attribute(model, "TimeLimit", 60.0)
optimize!(model);

### Plot test, de la merde.

In [42]:
fire_coords = []
for (k, coord) in enumerate(grouped_fires)
    push!(fire_coords,(lon=coord[1,:longitude],lat=coord[1,:latitude]))
end

In [73]:
function plot_fire_movements(fire_coords, X, Y, assignments)
    plotly()  # Set the backend to Plotly for interactive plotting
    p = plot()

    # Plot each fire location
    for (k, coord) in enumerate(fire_coords)
        #println((k, coord))
        scatter!([coord.lon], [coord.lat], label="Fire $k")
    end
    println("ok")
    # Plot initial assignments on day 1
    for (g, group_assignments) in enumerate(eachrow(assignments))
        for (k, assigned) in enumerate(group_assignments)
            if assigned == 1
                # Mark the assignment of group g to fire k
                scatter!([fire_coords[k].lon], [fire_coords[k].lat], 
                         label="Group $g assigned to Fire $k on Day 1")
            end
        end
    end
    println("ok")
    # Plot movements for each day
    for t in 2:3
        for g in 1:n_groups
            for k1 in 1:n_fires
                for k2 in 1:n_fires
                    #println(g, k1, k2, t)
                    if Y[g, k1, k2, t] == 1
                        
                        # Draw a line representing movement from k1 to k2
                        plot!([fire_coords[k1].lon, fire_coords[k2].lon], 
                              [fire_coords[k1].lat, fire_coords[k2].lat], 
                              label="Day $t Movement", arrow=true)
                        
                    end
                end
            end
        end
    end
    println("ok")
    #println("a")
    return p
end

# Call the function with the fire coordinates and X, Y1 matrices
p_ = plot_fire_movements(fire_coords, X_current, Y_current, assignements)

ok


LoadError: InterruptException:

In [22]:
fire_demand = sum(demand[k] for k=1:n_fires)/n_fires;

In [19]:
#n_groups = ceil(Int, n_fighters / 17)  # Calculate the number of groups 1000
n_groups = 100
model = Model(Gurobi.Optimizer)

# Redefine the variables with groups instead of individual fighters
@variable(model, X[g=1:n_groups, k=1:n_fires, t=1:3], binary = true)
@variable(model, Y[g=1:n_groups, k1=1:n_fires, k2=1:n_fires, t=2:3], binary = true)

# New variables for positive parts of costs
@variable(model, pos_cost[k=1:n_fires, t=1:3] >= 0)

# Constraints for new variables
@constraint(model, pos_cost_con[k=1:n_fires[1], t=1:3],
            pos_cost[k, t] >= fire_demand - sum(sum(X[g,k,t_] for g=1:n_groups) for t_=1:t))

# Adjusted objective function
@objective(model, Min, sum(pos_cost[k, t] for k=1:n_fires, t=1:3) + 
                        0.0000001 * sum(Y[g,k1,k2,t] * Distances[k1,k2] for g=1:n_groups, k1=1:n_fires, k2=1:n_fires, t=2:3))

# Adjusted constraints for groups
@constraint(model, constraint_fighters1[t=1:3, g=1:n_groups],
            sum(1[g,k,t] for k=1:n_fires) == 1)
@constraint(model, constraint_Y[t=2:3, k1=1:n_fires, k2=1:n_fires, g=1:n_groups],
            X[g,k1,t-1] + X[g,k2,t] - 1 <= Y[g,k1,k2,t])

set_optimizer_attribute(model, "TimeLimit", 60.0)
optimize!(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Set parameter TimeLimit to value 60
Set parameter TimeLimit to value 60
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 520653 rows, 535653 columns and 1606653 nonzeros
Model fingerprint: 0x0a19ea68
Variable types: 153 continuous, 535500 integer (535500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-09, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 8e+02]
Found heuristic solution: objective 121110.00016
Presolve removed 13953 rows and 13953 columns (presolve time = 5s) ...
Presolve removed 515586 rows and 530436 columns (presolve time = 10s) ...
Presolve removed 515586 rows and 530436 columns
Presolve time: 10.27s
Presolved: 5067 rows, 5217 columns, 15345 nonzeros
Found 

In [21]:
X_Alex1 = JuMP.value.(X1);

### Change approach (use matrix M instead of binary matrices)

In [72]:
model = Model(Gurobi.Optimizer)

# Variable for the number of groups moving from fire k1 to fire k2 between days t and t+1
@variable(model, M[k1=1:n_fires, k2=1:n_fires, t=1:2] >= 0, Int)

@variable(model, pos_cost[k=1:n_fires, t=1:3] >= 0)

@constraint(model, pos_cost_con0[k=1:n_fires, t=1],
            pos_cost[k, t] >= demand[k] - initial_allocation[k])

@constraint(model, pos_cost_con1[k=1:n_fires, t=2],
                pos_cost[k, 2] >= demand[k] - (sum(M[:, k, 1]) - sum(M[k, :, 1])))
@constraint(model, pos_cost_con2[k=1:n_fires, t=3],
            pos_cost[k, 3] >= demand[k] - (sum(M[:, k, 2]) - sum(M[k, :, 2])))


# Constraints based on movements
# Example: Constraint to ensure that the number of groups at a fire does not exceed its demand
# This is a conceptual example and may need adjustment to fit your specific case
@constraint(model, fighters_constraints[t=2:3],
    sum(M[k, :, t-1]) - sum(M[:, k, t-1]) <= demand[k] for k in 1:n_fires), t in 2:3)

# Objective function (adjust as necessary)
@objective(model, Min, sum(pos_cost[k, t] for k=1:n_fires, t=1:3) + 
                        0.0000001 * sum(M[k1, k2, t] * Distances[k1, k2] for k1=1:n_fires, k2=1:n_fires, t=1:2))

# Other necessary constraints based on the movement logic

set_optimizer_attribute(model, "TimeLimit", 60.0)
optimize!(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22


┌ Warning: Axis contains one element: 1. If intended, you can safely ignore this warning. To explicitly pass the axis with one element, pass `[1]` instead of `1`.
└ @ JuMP.Containers ~/.julia/packages/JuMP/D44Aq/src/Containers/DenseAxisArray.jl:186
┌ Warning: Axis contains one element: 2. If intended, you can safely ignore this warning. To explicitly pass the axis with one element, pass `[2]` instead of `2`.
└ @ JuMP.Containers ~/.julia/packages/JuMP/D44Aq/src/Containers/DenseAxisArray.jl:186
┌ Warning: Axis contains one element: 3. If intended, you can safely ignore this warning. To explicitly pass the axis with one element, pass `[3]` instead of `3`.
└ @ JuMP.Containers ~/.julia/packages/JuMP/D44Aq/src/Containers/DenseAxisArray.jl:186


LoadError: LoadError: At In[72]:20: `@constraint(model, demand_constraints[k = 1:n_fires, t = 1:3], (sum(M[k, :, t - 1]) - sum(M[:, k, t - 1]) <= demand[k] for k = 1:n_fires, t = 2:3))`: Unsupported constraint expression: we don't know how to parse constraints containing expressions of type :generator.

If you are writing a JuMP extension, implement `parse_constraint_head(::Function, ::Val{:generator}, args...)
in expression starting at In[72]:20